In [74]:
import sklearn
from sklearn.metrics import mean_squared_error, r2_score
import datetime
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [75]:
train_df = pd.read_csv('./train.csv')
train_df

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


In [76]:
sample_df = pd.read_csv('./sample_submission.csv')
sample_df

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
...,...,...
1087,RD_F_J_20230327,0
1088,RD_F_J_20230328,0
1089,RD_F_J_20230329,0
1090,RD_F_J_20230330,0


In [77]:
test_df = pd.read_csv('./test.csv')
test_df

,ID,timestamp,item,corporation,location
0,TG_A_J_20230304,2023-03-04,TG,A,J
1,TG_A_J_20230305,2023-03-05,TG,A,J
2,TG_A_J_20230306,2023-03-06,TG,A,J
3,TG_A_J_20230307,2023-03-07,TG,A,J
4,TG_A_J_20230308,2023-03-08,TG,A,J
...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J
1088,RD_F_J_20230328,2023-03-28,RD,F,J
1089,RD_F_J_20230329,2023-03-29,RD,F,J
1090,RD_F_J_20230330,2023-03-30,RD,F,J


In [78]:
trade_df = pd.read_csv('./international_trade.csv')
trade_df

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
4,2019-01,방울다다기 양배추,0,0,7580,38,-38
...,...,...,...,...,...,...,...
1269,2023-02,포포(papaw)[파파야(papaya)],0,0,23830,71,-71
1270,2023-02,사과,135165,351,0,0,351
1271,2023-02,배,2206012,5411,1,0,5411
1272,2023-02,신 체리[프루너스 체라서스(Prunus cerasus)],5,0,0,0,0


In [79]:
train_df.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [80]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])

In [81]:
# timestamp : year / month / day 쪼개기

train_df['year'] = train_df['timestamp'].dt.year
train_df['month'] = train_df['timestamp'].dt.month
train_df['day'] = train_df['timestamp'].dt.day
train_df['dayofweek'] = train_df['timestamp'].dt.dayofweek  ## 무슨요일인지 확인 

# test_df['year'] = test_df['timestamp'].dt.year
# test_df['month'] = df['timestamp'].dt.month
# test_df['day'] = df['timestamp'].dt.day
# test_df['dayofweek'] = test_df['timestamp'].dt.dayofweek

train_df.head(3)

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,dayofweek
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,2
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,3


In [82]:
# item Encoding
#TG: 감귤 (1)
#BC: 브로콜리 (2)
#RD: 무 (3)
#CR: 당근 (4)
#CB: 양배추 (5)
def get_item(x):
    if x == 'TG':
        return 1
    elif x == 'BC':
        return 2
    elif x == 'RD':
        return 3
    elif x == 'CR':
        return 4
    elif x == 'CB':
        return 5
        

In [83]:
#corporation Encoding
#A: 1
#B: 2
#C: 3
#D: 4
#E: 5
#F: 6
def get_cor(x):
    if x == 'A':
        return 1
    elif x == 'B':
        return 2
    elif x == 'C':
        return 3
    elif x == 'D':
        return 4
    elif x == 'E':
        return 5
    elif x == 'F':
        return 6
        

In [84]:
#location Encoding
#J: 1
#S: 2

def get_loc(x):
    if x == 'J':
        return 1
    elif x == 'S':
        return 2

In [85]:
train_df['item'].apply(get_item)
train_df['corporation'].apply(get_cor)
train_df['location'].apply(get_loc)

train_df['item_en'] = train_df['item'].apply(get_item)
train_df['cor_en'] = train_df['corporation'].apply(get_cor)
train_df['loc_en'] = train_df['location'].apply(get_loc)
train_df.head(3)

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,dayofweek,item_en,cor_en,loc_en
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,1,1
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,2,1,1,1
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,3,1,1,1


In [86]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
X = train_df[['supply(kg)','year','month','day','dayofweek','item_en','cor_en','loc_en']]
y = train_df[['price(원/kg)']]

In [87]:
model.fit(X,y)

LinearRegression()

In [88]:
y_pred_price = model.predict(X)
y_pred_price[:10]

array([[3077.35569431],
       [2923.51194328],
       [2714.95856507],
       [2593.25483545],
       [2432.77381476],
       [2308.13693918],
       [3211.81545682],
       [3074.23987764],
       [2918.32875073],
       [2771.74248213]])

In [89]:
train_df.head(10)

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,dayofweek,item_en,cor_en,loc_en
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,1,1
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,2,1,1,1
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,3,1,1,1
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,4,1,1,1
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,5,1,1,1
5,TG_A_J_20190106,2019-01-06,TG,A,J,0.0,0.0,2019,1,6,6,1,1,1
6,TG_A_J_20190107,2019-01-07,TG,A,J,44995.0,1474.0,2019,1,7,0,1,1,1
7,TG_A_J_20190108,2019-01-08,TG,A,J,26975.0,1326.0,2019,1,8,1,1,1,1
8,TG_A_J_20190109,2019-01-09,TG,A,J,29265.0,1428.0,2019,1,9,2,1,1,1
9,TG_A_J_20190110,2019-01-10,TG,A,J,21226.0,1433.0,2019,1,10,3,1,1,1


In [90]:
mean_squared_error(train_df['price(원/kg)'],y_pred_price)

2915446.7109898357